Importing LIBS

In [1]:
import os
import glob
import pandas as pd
from PIL import Image
import shutil
from ultralytics import YOLO
import cv2
import torch

Initializing VARS

In [3]:
""" import os
import glob

# Define the base directories for images and labels
base_image_dir = 'dataset/images'
base_label_dir = 'dataset/labels'

# Define the subdirectories for train, test, and validation
subdirs = ['train', 'test', 'validation']

# Function to remove files of class 2 (blink)
def remove_blink_files(base_dir):
    for subdir in subdirs:
        image_dir = os.path.join(base_dir, subdir)
        label_dir = os.path.join(base_label_dir, subdir)

        # Get all label files in the subdirectory
        label_files = glob.glob(os.path.join(label_dir, '*.txt'))

        for label_file in label_files:
            with open(label_file, 'r') as file:
                lines = file.readlines()

            # Check if any line in the label file corresponds to class 2 (blink)
            if any(line.startswith('2 ') for line in lines):
                # Get the corresponding image file (assuming same name but different extension)
                image_file = os.path.join(image_dir, os.path.basename(label_file).replace('.txt', '.jpg'))

                # Remove the image and label files
                os.remove(label_file)
                os.remove(image_file)

                print(f'Removed {label_file} and {image_file}')

# Remove blink files from images and labels
remove_blink_files(base_image_dir)
remove_blink_files(base_label_dir)
 """

In [2]:
""" imgs_base_path = 'Data/Original'
annotations_base_path = 'Annotation Subset'
imgs_base_path = 'Data/Original'
output_dir = 'dataset'
global images_output_dir_train, labels_output_dir_train, images_output_dir_valid, labels_output_dir_valid, images_output_dir_test, labels_output_dir_test
images_output_dir_train = os.path.join(output_dir, 'images/train')
labels_output_dir_train = os.path.join(output_dir, 'labels/train')
images_output_dir_valid = os.path.join(output_dir, 'images/valid')
labels_output_dir_valid = os.path.join(output_dir, 'labels/valid')
images_output_dir_test = os.path.join(output_dir, 'images/test')
labels_output_dir_test = os.path.join(output_dir, 'labels/test')
os.makedirs(images_output_dir_train, exist_ok=True)
os.makedirs(labels_output_dir_train, exist_ok=True)
os.makedirs(images_output_dir_valid, exist_ok=True)
os.makedirs(labels_output_dir_valid, exist_ok=True)
os.makedirs(images_output_dir_test, exist_ok=True)
os.makedirs(labels_output_dir_test, exist_ok=True)
image_paths = []
pupil_boxes = []
pupils_boxes = []
blink_boxes = [] """

" imgs_base_path = 'Data/Original'\nannotations_base_path = 'Annotation Subset'\nimgs_base_path = 'Data/Original'\noutput_dir = 'dataset'\nglobal images_output_dir_train, labels_output_dir_train, images_output_dir_valid, labels_output_dir_valid, images_output_dir_test, labels_output_dir_test\nimages_output_dir_train = os.path.join(output_dir, 'images/train')\nlabels_output_dir_train = os.path.join(output_dir, 'labels/train')\nimages_output_dir_valid = os.path.join(output_dir, 'images/valid')\nlabels_output_dir_valid = os.path.join(output_dir, 'labels/valid')\nimages_output_dir_test = os.path.join(output_dir, 'images/test')\nlabels_output_dir_test = os.path.join(output_dir, 'labels/test')\nos.makedirs(images_output_dir_train, exist_ok=True)\nos.makedirs(labels_output_dir_train, exist_ok=True)\nos.makedirs(images_output_dir_valid, exist_ok=True)\nos.makedirs(labels_output_dir_valid, exist_ok=True)\nos.makedirs(images_output_dir_test, exist_ok=True)\nos.makedirs(labels_output_dir_test, ex

GETTING DATA

In [3]:
""" # Directory containing the annotation files and images
annotation_dir_lst = ['eyes.v1i.yolov8/train/labels','eyes.v1i.yolov8/test/labels','eyes.v1i.yolov8/valid/labels']

# Iterate over all annotation files
for annotation_dir in annotation_dir_lst:
    image_dir = 'eyes.v1i.yolov8' + '/' +annotation_dir.split('/')[1] + '/' + 'images'
    for annotation_file in os.listdir(annotation_dir):
        if annotation_file.endswith('.txt'):
            annotation_path = os.path.join(annotation_dir, annotation_file)
            image_path = os.path.join(image_dir, annotation_file.replace('.txt', '.jpg'))

            with open(annotation_path, 'r') as file:
                lines = file.readlines()

            modified_lines = []
            contains_zero = False

            for line in lines:
                parts = line.strip().split()
                class_id = int(parts[0])

                if class_id == 0:
                    parts[0] = '2'
                    contains_zero = True

                modified_lines.append(' '.join(parts))

        # Delete the file and the corresponding image if it does not contain class 0 or 5
        if contains_zero:
            with open(annotation_path, 'w') as file:
                file.write('\n'.join(modified_lines))
        else:
            os.remove(annotation_path)
            if os.path.exists(image_path):
                os.remove(image_path) """

" # Directory containing the annotation files and images\nannotation_dir_lst = ['eyes.v1i.yolov8/train/labels','eyes.v1i.yolov8/test/labels','eyes.v1i.yolov8/valid/labels']\n\n# Iterate over all annotation files\nfor annotation_dir in annotation_dir_lst:\n    image_dir = 'eyes.v1i.yolov8' + '/' +annotation_dir.split('/')[1] + '/' + 'images'\n    for annotation_file in os.listdir(annotation_dir):\n        if annotation_file.endswith('.txt'):\n            annotation_path = os.path.join(annotation_dir, annotation_file)\n            image_path = os.path.join(image_dir, annotation_file.replace('.txt', '.jpg'))\n\n            with open(annotation_path, 'r') as file:\n                lines = file.readlines()\n\n            modified_lines = []\n            contains_zero = False\n\n            for line in lines:\n                parts = line.strip().split()\n                class_id = int(parts[0])\n\n                if class_id == 0:\n                    parts[0] = '2'\n                    con

In [4]:
""" def parse_data(file_path):
    image_paths = []
    pupils_boxes = []

    with open(file_path, 'r') as file:
        lines = file.readlines()
        
        for line in lines:
            parts = line.strip().split()
            
            # The image path is the first part
            image_path = parts[0]
            image_paths.append(file_path.split('\\')[1].split('.')[0] + '/' + image_path)
            
            # The remaining parts are bounding box coordinates
            bboxes = [int(x) for x in parts[1:]]
            pupils_boxes.append(bboxes[12:])
    
    return image_paths, pupils_boxes

def get_text_files_in_directory(directory_path):
    # Use glob to find all text files
    text_files = glob.glob(os.path.join(directory_path, '*.txt'))
    return text_files


txt_files = get_text_files_in_directory(annotations_base_path)

for file in txt_files:
    img_paths, pupils_b = parse_data(file)
    image_paths.extend(img_paths)
    pupils_boxes.extend(pupils_b)

# Example: How to pass to model training
# Create a dictionary or any other data structure you see fit
data = {
    'image_paths': image_paths,
    'pupils_boxes': pupils_boxes
}
df = pd.DataFrame(data) """

" def parse_data(file_path):\n    image_paths = []\n    pupils_boxes = []\n\n    with open(file_path, 'r') as file:\n        lines = file.readlines()\n        \n        for line in lines:\n            parts = line.strip().split()\n            \n            # The image path is the first part\n            image_path = parts[0]\n            image_paths.append(file_path.split('\\')[1].split('.')[0] + '/' + image_path)\n            \n            # The remaining parts are bounding box coordinates\n            bboxes = [int(x) for x in parts[1:]]\n            pupils_boxes.append(bboxes[12:])\n    \n    return image_paths, pupils_boxes\n\ndef get_text_files_in_directory(directory_path):\n    # Use glob to find all text files\n    text_files = glob.glob(os.path.join(directory_path, '*.txt'))\n    return text_files\n\n\ntxt_files = get_text_files_in_directory(annotations_base_path)\n\nfor file in txt_files:\n    img_paths, pupils_b = parse_data(file)\n    image_paths.extend(img_paths)\n    pupil

SHOWING DATA

In [5]:
#df.head(5)

CONVERTION TO YOLO FORMAT

In [6]:
""" # Function to convert bounding box to YOLO format
def convert_bbox_to_yolo_format(img_width, img_height, bbox):
    x_min, y_min, x_max, y_max = bbox
    x_min = max(0, min(x_min, img_width - 1))
    y_min = max(0, min(y_min, img_height - 1))
    x_max = max(0, min(x_max, img_width - 1))
    y_max = max(0, min(y_max, img_height - 1))
    x_center = (x_min + x_max) / 2.0
    y_center = (y_min + y_max) / 2.0
    
    # Calculate width and height
    width = x_max - x_min
    height = y_max - y_min
    
    # Normalize coordinates
    x_center /= img_width
    y_center /= img_height
    width /= img_width
    height /= img_height
    
    return x_center, y_center, width, height


def create_yolo_annotation_files(data, images_base_path):
    for i, row in data.iterrows():
        img_path = images_base_path + '/' + row['image_paths']
        if i % 8 == 0:
            images_output_dir = images_output_dir_valid
            labels_output_dir = labels_output_dir_valid
        elif i % 10 == 0:
            images_output_dir = images_output_dir_test
            labels_output_dir = labels_output_dir_test
        else:
            images_output_dir = images_output_dir_train
            labels_output_dir = labels_output_dir_train
        new_filename = img_path.replace('/', '_')
        dest_img_path = images_output_dir + '/' + new_filename
        shutil.copyfile(img_path, dest_img_path)
        img = Image.open(img_path)
        img_width, img_height = img.size
        
        bboxes = [
            (row['pupils_boxes'], 1)       # class_id for pupil is 0
        ]
        
        annotation_lines = []
        for bbox, class_id in bboxes:
            center_x, center_y, width, height = convert_bbox_to_yolo_format(img_width, img_height, bbox)
            annotation_lines.append(f"{class_id} {center_x} {center_y} {width} {height}")
            nfile = new_filename.split(".")[0]
        annotation_file_path = labels_output_dir + '/' + f"{nfile}.txt"

        # Ensure the directory exists
        os.makedirs(os.path.dirname(annotation_file_path), exist_ok=True)
        #print(annotation_file_path)
        with open(annotation_file_path, 'w') as f:
            f.write("\n".join(annotation_lines))

create_yolo_annotation_files(df, imgs_base_path) """

' # Function to convert bounding box to YOLO format\ndef convert_bbox_to_yolo_format(img_width, img_height, bbox):\n    x_min, y_min, x_max, y_max = bbox\n    x_min = max(0, min(x_min, img_width - 1))\n    y_min = max(0, min(y_min, img_height - 1))\n    x_max = max(0, min(x_max, img_width - 1))\n    y_max = max(0, min(y_max, img_height - 1))\n    x_center = (x_min + x_max) / 2.0\n    y_center = (y_min + y_max) / 2.0\n    \n    # Calculate width and height\n    width = x_max - x_min\n    height = y_max - y_min\n    \n    # Normalize coordinates\n    x_center /= img_width\n    y_center /= img_height\n    width /= img_width\n    height /= img_height\n    \n    return x_center, y_center, width, height\n\n\ndef create_yolo_annotation_files(data, images_base_path):\n    for i, row in data.iterrows():\n        img_path = images_base_path + \'/\' + row[\'image_paths\']\n        if i % 8 == 0:\n            images_output_dir = images_output_dir_valid\n            labels_output_dir = labels_outpu

In [7]:
""" # Define a function to convert YOLO format to corner coordinates
def yolo_to_coords(annotation, image_shape):
    x_center, y_center, width, height = annotation[1:]
    image_width = image_shape[1]
    image_height = image_shape[0]
    x_center *= image_width
    y_center *= image_height
    width *= image_width
    height *= image_height
    
    # Calculate corner coordinates
    x_min = x_center - (width / 2)
    y_min = y_center - (height / 2)
    x_max = x_center + (width / 2)
    y_max = y_center + (height / 2)
    print(x_min, y_min, x_max, y_max)
    return x_min, y_min, x_max, y_max

paths = df['image_paths'].to_list()
for i in range(20):
    image = cv2.imread('Data/Original' + '/' +paths[i]) 
    print(paths[i])
    class_names = {
        1: 'pupils'
    }
    image_shape = image.shape
    # Convert YOLO annotations to corner coordinates
    p_b = df['pupils_boxes'].to_list()[i]
    annotations = [
        [1, p_b[0], p_b[1], p_b[2], p_b[3]]
    ]
    bounding_boxes = [annotation[1:] for annotation in annotations]

    # Draw bounding boxes on the image
    for box, annotation in zip(bounding_boxes, annotations):
        class_id = annotation[0]
        class_name = class_names[class_id]
        box = list(map(int, box))
        print(box)
        cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)
        cv2.putText(image, class_name, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the image
    cv2.imshow('Image with Bounding Boxes', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
 """

" # Define a function to convert YOLO format to corner coordinates\ndef yolo_to_coords(annotation, image_shape):\n    x_center, y_center, width, height = annotation[1:]\n    image_width = image_shape[1]\n    image_height = image_shape[0]\n    x_center *= image_width\n    y_center *= image_height\n    width *= image_width\n    height *= image_height\n    \n    # Calculate corner coordinates\n    x_min = x_center - (width / 2)\n    y_min = y_center - (height / 2)\n    x_max = x_center + (width / 2)\n    y_max = y_center + (height / 2)\n    print(x_min, y_min, x_max, y_max)\n    return x_min, y_min, x_max, y_max\n\npaths = df['image_paths'].to_list()\nfor i in range(20):\n    image = cv2.imread('Data/Original' + '/' +paths[i]) \n    print(paths[i])\n    class_names = {\n        1: 'pupils'\n    }\n    image_shape = image.shape\n    # Convert YOLO annotations to corner coordinates\n    p_b = df['pupils_boxes'].to_list()[i]\n    annotations = [\n        [1, p_b[0], p_b[1], p_b[2], p_b[3]]\n

In [8]:
# Load a model
model = YOLO('last.pt')  # load a pre-trained model (smallest version)

In [9]:
# Train the model
model.train(
    data='dataset.yaml',  # path to the dataset yaml file
    epochs=70,  # number of epochs to train
    imgsz=640,  # image size,
    batch=32
)
# Save the model to a specified path
model.save('pupil_detector.pt')

New https://pypi.org/project/ultralytics/8.2.49 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.38  Python-3.9.0 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=last.pt, data=dataset.yaml, epochs=70, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, sa

train: Scanning E:\Previous work\deep learning\MPIGazz\MPIIGaze\MPIIGaze\dataset\labels\train.cache... 25159 images, 106 backgrounds, 4295 corrupt: 100%|██████████| 25159/25159 [00:00<?, ?it/s]

train: WARNING  E:\Previous work\deep learning\MPIGazz\MPIIGaze\MPIIGaze\dataset\images\train\Data_Original_p00_day01_0048.jpg: ignoring corrupt image/label: negative label values [ -0.0083333]
train: WARNING  E:\Previous work\deep learning\MPIGazz\MPIIGaze\MPIIGaze\dataset\images\train\Data_Original_p00_day01_0117.jpg: ignoring corrupt image/label: negative label values [ -0.0097222]
train: WARNING  E:\Previous work\deep learning\MPIGazz\MPIIGaze\MPIIGaze\dataset\images\train\Data_Original_p00_day01_0219.jpg: ignoring corrupt image/label: negative label values [  -0.011111]
train: WARNING  E:\Previous work\deep learning\MPIGazz\MPIIGaze\MPIIGaze\dataset\images\train\Data_Original_p00_day01_0291.jpg: ignoring corrupt image/label: negative label values [  -0.011111]
train: WARNING  E:\Previous work\deep learning\MPIGazz\MPIIGaze\MPIIGaze\dataset\images\train\Data_Original_p00_day01_0320.jpg: ignoring corrupt image/label: negative label values [  -0.013889]
train: WARNING  E:\Previous wo


val: Scanning E:\Previous work\deep learning\MPIGazz\MPIIGaze\MPIIGaze\dataset\labels\valid.cache... 2377 images, 0 backgrounds, 657 corrupt: 100%|██████████| 2377/2377 [00:00<?, ?it/s]

val: WARNING  E:\Previous work\deep learning\MPIGazz\MPIIGaze\MPIIGaze\dataset\images\valid\Data_Original_p00_day02_0017.jpg: ignoring corrupt image/label: negative label values [  -0.011111]
val: WARNING  E:\Previous work\deep learning\MPIGazz\MPIIGaze\MPIIGaze\dataset\images\valid\Data_Original_p00_day02_0705.jpg: ignoring corrupt image/label: negative label values [  -0.020833]
val: WARNING  E:\Previous work\deep learning\MPIGazz\MPIIGaze\MPIIGaze\dataset\images\valid\Data_Original_p00_day03_0470.jpg: ignoring corrupt image/label: negative label values [  -0.016667]
val: WARNING  E:\Previous work\deep learning\MPIGazz\MPIIGaze\MPIIGaze\dataset\images\valid\Data_Original_p00_day04_0125.jpg: ignoring corrupt image/label: negative label values [ -0.0083333]
val: WARNING  E:\Previous work\deep learning\MPIGazz\MPIIGaze\MPIIGaze\dataset\images\valid\Data_Original_p00_day04_0608.jpg: ignoring corrupt image/label: negative label values [ -0.0041667]
val: WARNING  E:\Previous work\deep lear

...

      70/70       4.4G     0.8397     0.3636     0.9506         31        640: 100%|██████████| 652/652 [02:25<00:00,  4.48it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 27/27 [00:08<00:00,  3.24it/s]

                   all       1720       1720      0.907      0.774      0.825      0.537



70 epochs completed in 4.549 hours.
Optimizer stripped from runs\detect\train9\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train9\weights\best.pt, 6.2MB

Validating runs\detect\train9\weights\best.pt...
Ultralytics YOLOv8.2.38  Python-3.9.0 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 27/27 [00:09<00:00,  2.85it/s]


                   all       1720       1720      0.907      0.774      0.826      0.537
                 Pupil        524        524      0.893      0.924       0.93      0.679
                Pupils        675        675      0.836      0.407      0.554      0.274
                 Blink        521        521      0.993      0.992      0.994      0.658
Speed: 0.2ms preprocess, 1.2ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect\train9


In [10]:
# Evaluate the model
model.val()

Ultralytics YOLOv8.2.38  Python-3.9.0 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning E:\Previous work\deep learning\MPIGazz\MPIIGaze\MPIIGaze\dataset\labels\valid.cache... 2377 images, 0 backgrounds, 657 corrupt: 100%|██████████| 2377/2377 [00:00<?, ?it/s]

val: WARNING  E:\Previous work\deep learning\MPIGazz\MPIIGaze\MPIIGaze\dataset\images\valid\Data_Original_p00_day02_0017.jpg: ignoring corrupt image/label: negative label values [  -0.011111]
val: WARNING  E:\Previous work\deep learning\MPIGazz\MPIIGaze\MPIIGaze\dataset\images\valid\Data_Original_p00_day02_0705.jpg: ignoring corrupt image/label: negative label values [  -0.020833]
val: WARNING  E:\Previous work\deep learning\MPIGazz\MPIIGaze\MPIIGaze\dataset\images\valid\Data_Original_p00_day03_0470.jpg: ignoring corrupt image/label: negative label values [  -0.016667]
val: WARNING  E:\Previous work\deep learning\MPIGazz\MPIIGaze\MPIIGaze\dataset\images\valid\Data_Original_p00_day04_0125.jpg: ignoring corrupt image/label: negative label values [ -0.0083333]
val: WARNING  E:\Previous work\deep learning\MPIGazz\MPIIGaze\MPIIGaze\dataset\images\valid\Data_Original_p00_day04_0608.jpg: ignoring corrupt image/label: negative label values [ -0.0041667]
val: WARNING  E:\Previous work\deep lear


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 54/54 [00:10<00:00,  5.09it/s]


                   all       1720       1720      0.905      0.776      0.829      0.538
                 Pupil        524        524      0.893      0.927      0.929      0.678
                Pupils        675        675      0.829      0.409      0.565      0.278
                 Blink        521        521      0.993      0.992      0.994      0.658
Speed: 0.3ms preprocess, 2.3ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\train92


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000020EC071F160>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          

yolov8 results

In [11]:
# Load YOLOv8 model
model = YOLO('pupil_detector.pt')  # You can replace 'yolov8n.pt' with your trained YOLOv8 model

# Path to your image
results = model('0203.jpg')
print(results)
# Access the original image
image = results[0].orig_img

# Access the boxes, labels, and confidence scores
boxes = results[0].boxes.xyxy  # Bounding boxes
labels = results[0].boxes.cls  # Class labels
confidences = results[0].boxes.conf  # Confidence scores
names = results[0].names  # Class names dictionary

# Iterate through each detected object and draw the bounding box with label
for box, label, confidence in zip(boxes, labels, confidences):
    x1, y1, x2, y2 = map(int, box)  # Convert box coordinates to integers
    class_name = names[int(label)]  # Get the class name
    label_text = f'{class_name} {confidence:.2f}'  # Label text with confidence

    # Draw the bounding box
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Draw the label
    cv2.putText(image, label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# Display the image with predictions
cv2.imshow('Predictions', image)
cv2.waitKey(0)
cv2.destroyAllWindows()



image 1/1 e:\Previous work\deep learning\MPIGazz\MPIIGaze\MPIIGaze\0203.jpg: 384x640 1 Pupils, 79.7ms
Speed: 1.6ms preprocess, 79.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'Pupil', 1: 'Pupils', 2: 'Blink'}
obb: None
orig_img: array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
